# Bitmask to polygonal annotation
This notebook contains functions that allow us to transform bitmasks to polygonal annotations in COCO format
## Main code (just for one instance per image, for more than one see bit_to_poly.ipynb)

In [3]:
from PIL import Image # (pip install Pillow)
import matplotlib.pyplot as plt
import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)
import cv2
import os
#
from imantics import Polygons, Mask                 # (pip install imantics)

In [2]:


# This can be any array
img_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_square/silhouettes/002-000_crop.png"
ground_truth_binary_mask = cv2.imread(img_path,0)
fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
polygons = Mask(fortran_ground_truth_binary_mask).polygons()

print(polygons.points)
print(polygons.segmentation)

[array([[600, 175],
       [599, 176],
       [596, 176],
       ...,
       [622, 176],
       [620, 176],
       [619, 175]])]
[[600, 175, 599, 176, 596, 176, 595, 177, 593, 177, 592, 178, 590, 178, 589, 179, 588, 179, 587, 180, 586, 180, 585, 181, 584, 181, 582, 183, 581, 183, 580, 184, 580, 185, 579, 186, 578, 186, 578, 187, 576, 189, 576, 191, 575, 192, 576, 193, 576, 197, 577, 198, 577, 199, 578, 200, 578, 201, 579, 202, 579, 213, 578, 214, 578, 221, 579, 222, 579, 225, 580, 226, 580, 227, 581, 228, 581, 229, 584, 232, 584, 233, 590, 239, 592, 239, 595, 242, 595, 246, 591, 250, 590, 250, 589, 251, 588, 251, 587, 252, 585, 252, 583, 254, 582, 254, 576, 260, 576, 261, 575, 262, 575, 263, 574, 264, 574, 266, 573, 267, 573, 275, 572, 276, 572, 278, 571, 279, 571, 281, 570, 282, 570, 284, 569, 285, 569, 286, 568, 287, 568, 313, 567, 314, 567, 334, 568, 335, 568, 343, 567, 344, 567, 352, 566, 353, 566, 357, 562, 361, 562, 362, 561, 363, 561, 364, 560, 365, 560, 367, 559, 368, 559, 371,

In [13]:
def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

In [19]:
import json
import numpy as np
from pycocotools import mask
from skimage import measure

def mask_to_annotation(img_path, img_id, img_cat, ann_id, is_crowd):
    ground_truth_binary_mask = cv2.imread(img_path,0)
    fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
    encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
    ground_truth_area = mask.area(encoded_ground_truth)
    ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)

    padded_binary_mask = np.pad(ground_truth_binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)

    annotation = {
            "segmentation": [],
            "area": ground_truth_area.tolist(),
            "iscrowd": is_crowd,
            "image_id": img_id,
            "bbox": ground_truth_bounding_box.tolist(),
            "category_id": img_cat,
            "id": ann_id
        }

    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, 0)   #tolerance = 0
        if len(contour) < 3:
            continue

        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        segmentation = [0 if i < 0 else i for i in segmentation]
        annotation["segmentation"].append(segmentation)
        
    
    return annotation

In [15]:
def images_to_annotations(dir_path, img_names):
    #Initializing images and annotations lists
    images=[]
    annotations = []
    img_names.sort()
    img_license = 0
    img_id=0
    ann_id=0
    is_crowd=0
           
    for img_name in img_names:
        
        img_path = os.path.join(dir_path, img_name)
        img = cv2.imread(img_path)
        img_size = img.shape
        
        img_cat = 1
        annotation = mask_to_annotation(img_path , img_id, img_cat, ann_id, is_crowd)
        annotations.append(annotation)
        ann_id+=1
            

        new_img={}
        new_img["license"] = img_license
        new_img["file_name"] = img_name.split(".")[0]+".jpg" #Changed to match the video images
        new_img["width"] = img_size[1]
        new_img["height"] = img_size[0]
        new_img["id"] = img_id
        images.append(new_img)

        # sub_masks = create_sub_masks()
        # for color, sub_mask in sub_masks.items():
        #     plt.imshow(sub_mask)
        #     plt.show()
        
        img_id+=1

        sys.stdout.write('\r'+  "Done: {}/{}".format(img_id,len(img_names)))
        sys.stdout.flush()

    return annotations,images





In [16]:
def create_annotation_dict(info, licenses, imgs,categories,anns):
    my_dict = {}
    my_dict["info"]= info
    my_dict["licenses"]= licenses
    my_dict["images"]=imgs
    my_dict["categories"]=categories
    my_dict["annotations"]=anns
    return my_dict

## Creating one dataset

In [47]:
dir_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/silhouettes"
# dir_path = '/home/josmar/proyectos/codes/datasets/casia_B1_silhouettes'
img_names=os.listdir(dir_path)
color2cat={"[255 255 255]": 1}
info = {
    "description":"Test Dataset",
    "url":"",
    "version":"0.1",
    "year":2020,
    "contributor":"Josmar Suarez",
    "date_created":"2020/07/14"
}

licenses = [{
        "url": "",
        "id": 0,
        "name": "Attribution-NonCommercial-ShareAlike License"
    }]
categories = [
    {
        "supercategory":"person",
        "id":1,
        "name":"person"
    }
]
anns,imgs = images_to_annotations(dir_path, img_names)
my_dict = create_annotation_dict(info, licenses, imgs,categories,anns)


Done: 300/300

## Creating train,val, test datasets

In [17]:
import random
#percent of ____ from 0 to 1
p_train = 0.8
p_val = 0.1
p_test = 0.1

#Reading and shuffling images list
dir_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_square/silhouettes"
all_images = os.listdir(dir_path)
random.shuffle(all_images)
# Length of the image list
n = len(all_images)
# Limits of the index
lim1 = int(p_train*n)
lim2 = lim1 + int(p_val*n)
# Creating lists for each value
train_l = all_images[0:lim1]
val_l = all_images[lim1:lim2]
test_l = all_images[lim2:n]
# Visualizing sizes
print(len(train_l),len(val_l),len(test_l))


9840 1230 1230


In [20]:
dir_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_square/silhouettes"
# dir_path = '/home/josmar/proyectos/codes/datasets/casia_B1_silhouettes'
all_dicts = [] 
datasets = [train_l,val_l,test_l]
titles = ["Ucb Gait Train","Ucb Gait Val", "Ucb Gait Test"]
index=0
for l in [train_l,val_l,test_l]:
    color2cat={"[255 255 255]": 1}
    info = {
        "description":titles[index],
        "url":"",
        "version":"0.1",
        "year":2020,
        "contributor":"Josmar Suarez",
        "date_created":"2020/10/28"
    }

    licenses = [{
            "url": "",
            "id": 0,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }]
    categories = [
        {
            "supercategory":"person",
            "id":1,
            "name":"person"
        }
    ]
    print("\n",titles[index])
    anns,imgs = images_to_annotations(dir_path, l)
    my_dict = create_annotation_dict(info, licenses, imgs,categories,anns)
    all_dicts.append(my_dict)
    index+=1


 Ucb Gait Train
Done: 9840/9840
 Ucb Gait Val
Done: 1230/1230
 Ucb Gait Test
Done: 1230/1230

In [21]:
import json 
titles = ["/home/josmar/proyectos/codes/datasets/ucb_gait_square/annotations/ucb_gait_train.json","/home/josmar/proyectos/codes/datasets/ucb_gait_square/annotations/ucb_gait_val.json", "/home/josmar/proyectos/codes/datasets/ucb_gait_square/annotations/ucb_gait_test.json"]
index=0
for my_dict in all_dicts:
    with open(titles[index], 'w') as fp:
        json.dump(my_dict, fp)
    index += 1

In [76]:
len(all_dicts[2]["annotations"])

30

## Saving the annotations in json format

In [6]:
import json 
with open('ucb_gait_poly_new.json', 'w') as fp:
    json.dump(my_dict, fp)

In [7]:
my_dict["images"][0]

{'license': 0,
 'file_name': '002-000.jpg',
 'width': 1280,
 'height': 720,
 'id': 0}

## Extracting all silhouetes and images in datasets
```
import glob 
import shutil

    
destination_path = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/silhouettes"
pattern = "/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/*/silhouette/*"  
for img in glob.glob(pattern):
    shutil.copy(img, destination_path)
```


## Reviewing dictionary

In [10]:
import json 
with open('/home/josmar/proyectos/codes/datasets/ucb_gait_square/annotations/ucb_gait_train.json') as f:
    data = json.load(f)

In [10]:
lengths=[]
for ann in data["annotations"]:
    l=len(data["annotations"][0]["segmentation"][0])
    lengths.append(l)
print(set(lengths))

{3368}


In [2]:
category_list = []
for ann in data["annotations"]:
    category_list.append(ann["category_id"])
print(set(category_list))

{1}


In [12]:
n_points = []
tiny = 0
tiny_th = 3
total_anns =len(data["annotations"])
for ann in data["annotations"]:
    ann_points = []
    for i in ann["segmentation"]:
        n=len(i)
        if n < tiny_th:
            tiny+=1
        ann_points.append(n)
    n_points.append(ann_points)
    
    sys.stdout.write('\r'+  "Done: {}/{}".format(ann["image_id"]+1,total_anns))
    sys.stdout.flush()
print("\nTotal_tiny", tiny)


Done: 9840/9840
Total_tiny 127


In [32]:
print

9840

In [20]:
data["annotations"][2].keys()

dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id'])